In [1]:
import os 

import numpy as np
import pandas as pd
import torch
import torch.nn as nn
import torch.functional as F
from tqdm import tqdm
from pytorch_pretrained_bert import BertTokenizer, BertModel

from glue_datasets import QNLIDataset
from metal import EndModel

Better speed can be achieved with apex installed from https://www.github.com/nvidia/apex.


### Preprocess data

In [2]:
model = 'bert-base-uncased' # also try bert-base-multilingual-cased (recommended)
tokenizer = BertTokenizer.from_pretrained('bert-base-uncased', do_lower_case=True)
src_path = os.path.join(os.environ['METALHOME'], 'data/QNLI/{}.tsv')
dataset = {}
for split in ['dev']: # 'train', 'test', 'dev']:
    dataset[split] = QNLIDataset(src_path.format(split), tokenizer, nb_classes=2)
    dataset[split].load_data()
    dataset[split].preprocess_data()

100%|██████████| 5266/5266 [00:07<00:00, 725.91it/s]


In [3]:
dataset['dev'].raw_data.head()

,question,sentence,label
index,,,
0,What came into force after the new constitutio...,"As of that day, the new constitution heralding...",entailment
1,What is the first major city in the stream of ...,The most important tributaries in this area ar...,not_entailment
2,What is the minimum required if you want to te...,In most provinces a second Bachelor's Degree s...,not_entailment
3,How was Temüjin kept imprisoned by the Tayichi...,The Tayichi'ud enslaved Temüjin (reportedly wi...,entailment
4,"What did Herr Gott, dich loben wir become know...","He paraphrased the Te Deum as ""Herr Gott, dich...",not_entailment


### Metal Model 

In [20]:
class BertEncoder(nn.Module):
    def __init__(self):
        super(BertEncoder, self).__init__()
        self.bert_model = BertModel.from_pretrained('bert-base-uncased')
        
    def forward(self, data):
        tokens, segments, masks = data
        # TODO: check if we should return all layers or just last hidden representation 
        _, hidden_layer = self.bert_model(tokens, segments, masks)
        return hidden_layer

In [21]:
encoder_module = BertEncoder()
end_model = EndModel(
    [768, 2],
    input_module=encoder_module,
    seed=123,
    use_cuda=False,
    skip_head=False,
    input_relu=False,
    input_batchnorm=False,
    verbose=False
)

### Train model

In [ ]:
end_model.train_model(dataset['dev'].get_dataloader(max_len=-1),
                      lr=0.01, l2=0.01, 
                      batch_size=256, 
                      n_epochs=5, checkpoint_metric='accuracy',
                      checkpoint_metric_mode='max',
                      verbose=True)

### Test model

In [ ]:
# Test end model
end_model.score(test_loader, metric=["accuracy", "precision", "recall", "f1"])